In [340]:
import pandas as pd
import numpy as np
import glob
import seaborn as sns
pd.set_option('display.max_columns', None)

### set folder

In [223]:
input = 'input//'
output = 'output//'
# os.mkdir(input)
# os.mkdir(output)

In [224]:
data = glob.glob(input+'*.txt')[-1]
df = pd.read_csv(data ,sep='\t',encoding='TIS-620')

In [225]:
df = df.dropna(subset=['FlagGB_DQ60'])
df.reset_index(drop=True,inplace=True)

## feature selection

All feature

In [ ]:
['WrongDept':'เคยผิดนัดชำระหนี้','LongTermSaving_More50K':'การออมทรัพย์ระยะยาวที่มียอดเงินฝากตั้งแต่ 5 หมื่น',
 'AccommodationOwner':'ที่พักอาศัย','AccommodationType':'ประเภทที่อยู่อาศัย','Age':'อายุ','BusinessLocation':'สถานที่',
 'Gender':'เพศ','Occupation':'ความมั่นคงของรายได้','Position':'ตำแหน่งหน้าที่การงาน(Job Position)','WorkExperience':'อายุพนักงาน'
 'CompanyAge':'อายุงานบริษัท','SalaryDeductionAgreement':'สินเชื่อที่มีข้อตกลงกับธนาคาร']

feature binning

In [57]:
['WrongDept':'เคยผิดนัดชำระหนี้','LongTermSaving_More50K':'การออมทรัพย์ระยะยาวที่มียอดเงินฝากตั้งแต่ 5 หมื่น',
 'AccommodationOwner':'ที่พักอาศัย','Age':'อายุ','BusinessLocation':'สถานที่',
 'Gender':'เพศ','Occupation':'ความมั่นคงของรายได้','Position':'ตำแหน่งหน้าที่การงาน(Job Position)','WorkExperience':'อายุพนักงาน',
 'CompanyAge':'อายุงานบริษัท','SalaryDeductionAgreement':'สินเชื่อที่มีข้อตกลงกับธนาคาร']

SyntaxError: invalid syntax (<ipython-input-57-fdc199090b5b>, line 1)

In [58]:
columns = ['WrongDept','LongTermSaving_More50K',
 'AccommodationOwner','Age''BusinessLocation',
 'Gender','Occupation','Position','WorkExperience',
 'CompanyAge','SalaryDeductionAgreement']

## Woe_transform

In [267]:
def transform(df):
    # df_new = pd.DataFrame()
    df_new = df
    #เคยผิดนัดชำระหนี้
    df_new['WrongDept : TDR และผ่อนเงินต้นพร้อมดอกเบี้ยมาแล้วตามเงื่อนไข 24 เดือน'] = np.where(
        df_new['WrongDept'].isin(['TDR และผ่อนเงินต้นพร้อมดอกเบี้ยมาแล้วตามเงื่อนไข 24 เดือน']), 1, 0)
    df_new['WrongDept : ผิดนัดชำระหนี้ไม่เกิน 30 วัน'] = np.where(
        df_new['WrongDept'].isin(['ผิดนัดชำระหนี้ไม่เกิน 30 วัน']), 1, 0)
    df_new['WrongDept : เคยผิดนัดชำระหนี้ตั้งแต่ 31 วัน ถึง 60 วัน'] = np.where(
        df_new['WrongDept'].isin(['เคยผิดนัดชำระหนี้ตั้งแต่ 31 วัน ถึง 60 วัน']), 1, 0)
    df_new['WrongDept : เคยผิดชำระหนี้มากกว่า 60 วัน'] = np.where(
        df_new['WrongDept'].isin(['เคยผิดชำระหนี้มากกว่า 60 วัน']), 1, 0)
    
    #	LongTermSaving_More50K
    df_new['LongTermSaving_More50K : มี'] = np.where(
        (df_new['LongTermSaving_More50K'] >= 50000), 1, 0)
    df_new['LongTermSaving_More50K : ไม่มี'] = np.where(
        (df_new['LongTermSaving_More50K'] < 50000), 1, 0)
    
    # AccommodationOwner
    df_new['AccommodationOwner : เป็นของตนเอง/คู่สมรส'] = np.where(
        df_new['AccommodationOwner'].isin(['เป็นของตนเอง/คู่สมรส']), 1, 0)
    df_new['AccommodationOwner : พักอาศัยอยู่กับบิดามารดา/ญาติ'] = np.where(
        df_new['AccommodationOwner'].isin(['พักอาศัยอยู่กับบิดามารดา/ญาติ']), 1, 0)
    df_new['AccommodationOwner : พักอาศัยอยู่กับผู้อื่น'] = np.where(
        df_new['AccommodationOwner'].isin(['พักอาศัยอยู่กับผู้อื่น']), 1, 0)
    df_new['AccommodationOwner : เช่า'] = np.where(
        df_new['AccommodationOwner'].isin(['เช่า']), 1, 0)
    df_new['AccommodationOwner : สถานที่อื่นๆ'] = np.where(
        df_new['AccommodationOwner'].isin(['สถานที่อื่นๆ']), 1, 0)

    #Age
    df_new['Age : GroupA <= 20'] = np.where((df_new.Age <= 20),1,0)
    df_new['Age : GroupB 21-23'] = np.where(((df_new.Age <= 23) & (df_new.Age > 21)),1,0)
    df_new['Age : GroupC 24-30'] = np.where(((df_new.Age <= 30) & (df_new.Age > 23)),1,0)
    df_new['Age : GroupD 31-40'] = np.where(((df_new.Age <= 40) & (df_new.Age > 30)),1,0)
    df_new['Age : GroupE 41-65'] = np.where(((df_new.Age <= 65) & (df_new.Age > 40)),1,0)
    df_new['Age : GroupF > 65'] = np.where((df_new.Age > 65),1,0)

    #BusinessLocation
    df_new['BusinessLocation : กรุงเทพฯเเละปริมณฑล'] = np.where(
        df_new['BusinessLocation'].isin(['กรุงเทพมหานคร', 'นครปฐม', 'นนทบุรี', 'สมุทรปราการ', 'ปทุมธานี', 'สมุทรสาคร']), 1, 0)
    df_new['BusinessLocation : อื่นๆ'] = np.where(~(df_new['BusinessLocation'].isin(['กรุงเทพมหานคร', 'นครปฐม', 'นนทบุรี',
                                    'สมุทรปราการ', 'ปทุมธานี', 'สมุทรสาคร'])) & (df_new['BusinessLocation'].notna()) ,1 ,0)
   
    #Occupation           
    df_new['Occupation : ข้าราชการ'] = np.where(
        df_new['Occupation'].isin(['ข้าราชการและลูกจ้างในหน่วยงานของรัฐ','ประกอบวิชาชีพเฉพาะ']), 1, 0)
    df_new['Occupation : พนักงานรัฐวิสาหกิจ'] = np.where(
        df_new['Occupation'].isin(['พนักงานและลูกจ้างในหน่วยงานรัฐวิสาหกิจ']), 1, 0)
    df_new['Occupation : พนักงานบริษัท'] = np.where(
        df_new['Occupation'].isin(['พนักงานและลูกจ้างในบริษัท']), 1, 0)
    df_new['Occupation : ผู้ประกอบการธุรกิจส่วนตัว (จดทะเบียน)'] = np.where(
        df_new['Occupation'].isin(['ผู้ประกอบกิจการ / เจ้าของกิจการ']), 1, 0)
    df_new['Occupation : ผู้ประกอบการธุรกิจส่วนตัว (ไม่จดทะเบียน)'] = np.where(
        df_new['Occupation'].isin(['ประกอบอาชีพอิสระ']), 1, 0)
    df_new['Occupation : ผู้ประกอบการธุรกิจส่วนตัว (ไม่จดทะเบียน)'] = np.where(
        df_new['Occupation'].isin(['ประกอบอาชีพอิสระ']), 1, 0)
    df_new['Occupation : อื่นๆ'] = np.where(
        df_new['Occupation'].isin(['อื่น ๆ โปรดระบุ']), 1, 0)

    #Position
    df_new['Position : บริหารระดับสูง'] = np.where(
        df_new['Position'].isin(['บริหารระดับสูง']), 1, 0)
    df_new['Position : ผู้บริหารระดับกลาง'] = np.where(
        df_new['Position'].isin(['ผู้บริหารระดับกลาง']), 1, 0)
    df_new['Position : ผู้บริหารระดับต้น'] = np.where(
        df_new['Position'].isin(['ผู้บริหารระดับต้น']), 1, 0)
    df_new['Position : พนักงานเจ้าหน้าที่'] = np.where(
        df_new['Position'].isin(['พนักงานเจ้าหน้าที่']), 1, 0)
    df_new['Position : อื่นๆ'] = np.where(
        df_new['Position'].isin(['อื่นๆ']), 1, 0)  

    #WorkExperience
    df_new['WorkExperience : GroupA < 1'] = np.where((df_new.WorkExperience < 1),1,0)
    df_new['WorkExperience : GroupB 1-3'] = np.where(((df_new.WorkExperience <= 3) & (df_new.WorkExperience >= 1)),1,0)
    df_new['WorkExperience : GroupC 3-5'] = np.where(((df_new.WorkExperience <= 5) & (df_new.WorkExperience > 3)),1,0)
    df_new['WorkExperience : GroupF > 5'] = np.where((df_new.WorkExperience > 5),1,0)
    
    #CompanyAge
    df_new['CompanyAge : GroupA < 2'] = np.where((df_new.CompanyAge < 2),1,0)
    df_new['CompanyAge : GroupB 2-5'] = np.where(((df_new.CompanyAge <= 5) & (df_new.CompanyAge >= 2)),1,0)
    df_new['CompanyAge : GroupC >5-7'] = np.where(((df_new.CompanyAge <= 7) & (df_new.CompanyAge > 5)),1,0)
    df_new['CompanyAge : GroupF > 7'] = np.where((df_new.CompanyAge > 7),1,0)

    #SalaryDeductionAgreement
    df_new['SalaryDeductionAgreement : หน่วยงานมีข้อตกลงหักเงินเดือนชำระหนี้ให้ธนาคารเป็นลำดับแรกก่อนเจ้าหนี้รายอื่น'] = np.where(
        df_new['SalaryDeductionAgreement'].isin(['หน่วยงานมีข้อตกลงหักเงินเดือนชำระหนี้ให้ธนาคารเป็นลำดับแรกก่อนเจ้าหนี้รายอื่น']), 1, 0)
    df_new['SalaryDeductionAgreement : หน่วยงานมีข้อตกลงหักเงินเดือน'] = np.where(
        df_new['SalaryDeductionAgreement'].isin(['หน่วยงานมีข้อตกลงหักเงินเดือน']), 1, 0)
    df_new['SalaryDeductionAgreement : หน่วยงานมีข้อตกลงนำเงินสะสมเข้าบัญชีกรณีออกจากงาน หรือหน่วยงานชำระหนี้แทนถ้าผิดนัดชำระหนี้'] = np.where(
        df_new['SalaryDeductionAgreement'].isin(['หน่วยงานมีข้อตกลงนำเงินสะสมเข้าบัญชีกรณีออกจากงาน หรือหน่วยงานชำระหนี้แทนถ้าผิดนัดชำระหนี้']), 1, 0)
    df_new['SalaryDeductionAgreement : โครงการมีข้อตกลงรับซื้อคืนกรณีผู้ขอสินเชื่อผิดนัดชำระหนี้'] = np.where(
        df_new['SalaryDeductionAgreement'].isin(['โครงการมีข้อตกลงรับซื้อคืนกรณีผู้ขอสินเชื่อผิดนัดชำระหนี้']), 1, 0)
    df_new['SalaryDeductionAgreement : ไม่มีโครงการ'] = np.where(
        df_new['SalaryDeductionAgreement'].isin(['ไม่มีโครงการ']), 1, 0)    
    return df_new

## for woe cal

In [332]:
# df main ที่สร้างขึ้น ,df คือ all data ,
def process_woe(df_main, df, col, col_gb):   
    df_woe = pd.concat([ df_main, 
                        df[[col]].groupby(col)[col].count(), 
                        df[df[col_gb] == 'Good'].groupby(col)[col].count(),
                        df[df[col_gb] == 'Bad'].groupby(col)[col].count()], axis=1).fillna(0)
    
    df_woe.columns = ['n_obs', 'n_good', 'n_bad']
    return df_woe

def WOE_cal(df):
        df['prop_n_obs'] = df['n_obs']/df['n_obs'].sum()
        df['prop_bad'] = df['n_bad']/df['n_obs'].sum()
        df['prop_good'] = (1 - df['prop_bad'])
        df['prop_n_good'] = df['n_good']/df['n_good'].sum()
        df['prop_n_bad'] = df['n_bad']/df['n_bad'].sum()
        df['WOE'] = np.log(df['prop_n_good']/df['prop_n_bad'])
        df['IV'] = (df['prop_n_good']-df['prop_n_bad'])*df['WOE']
        return df

In [345]:
df.columns.values

array(['ApplicationNo', 'Month', 'ID', 'BusinessGroup', 'BusinessType',
       'LongTermSaving_More50K', 'Age', 'BusinessLocation',
       'InsuranceDeptPayment', 'Gender', 'AccommodationOwner',
       'LengthofStay', 'WrongDept', 'Religion', 'AccommodationType',
       'NumofEmployee', 'NumofSelfEmployee', 'WorkExperience',
       'SalaryDeductionAgreement', 'Occupation', 'Position',
       'OccupationType', 'CompanyAge', 'FlagGB_DQ60', 'FlagGB_DQ90',
       'WrongDept : TDR และผ่อนเงินต้นพร้อมดอกเบี้ยมาแล้วตามเงื่อนไข 24 เดือน',
       'WrongDept : ผิดนัดชำระหนี้ไม่เกิน 30 วัน',
       'WrongDept : เคยผิดนัดชำระหนี้ตั้งแต่ 31 วัน ถึง 60 วัน',
       'WrongDept : เคยผิดชำระหนี้มากกว่า 60 วัน',
       'LongTermSaving_More50K : มี', 'LongTermSaving_More50K : ไม่มี',
       'AccommodationOwner : เป็นของตนเอง/คู่สมรส',
       'AccommodationOwner : พักอาศัยอยู่กับบิดามารดา/ญาติ',
       'AccommodationOwner : พักอาศัยอยู่กับผู้อื่น',
       'AccommodationOwner : เช่า', 'AccommodationOwner :

In [333]:
#เคยผิดนัดชำระหนี้
data = {'WrongDept': ['ผิดนัดชำระหนี้ไม่เกิน 30 วัน', 'TDR และผ่อนเงินต้นพร้อมดอกเบี้ยมาแล้วตามเงื่อนไข 24 เดือน', 'เคยผิดนัดชำระหนี้ตั้งแต่ 31 วัน ถึง 60 วัน', 'เคยผิดชำระหนี้มากกว่า 60 วัน']}
df_main_WrongDept = pd.DataFrame(data=data).set_index('WrongDept')
WrongDept = process_woe(df_main=df_main_WrongDept, df=df, col='WrongDept', col_gb='FlagGB_DQ60')
WOE_cal(WrongDept)

,n_obs,n_good,n_bad,prop_n_obs,prop_bad,prop_good,prop_n_good,prop_n_bad,WOE,IV
ผิดนัดชำระหนี้ไม่เกิน 30 วัน,3477.0,3452.0,25.0,0.983871,0.007074,0.992926,0.983756,1.0,-0.016377,0.000266
TDR และผ่อนเงินต้นพร้อมดอกเบี้ยมาแล้วตามเงื่อนไข 24 เดือน,3.0,3.0,0.0,0.000849,0.000000,1.000000,0.000855,0.0,inf,inf
เคยผิดนัดชำระหนี้ตั้งแต่ 31 วัน ถึง 60 วัน,54.0,54.0,0.0,0.015280,0.000000,1.000000,0.015389,0.0,inf,inf
เคยผิดชำระหนี้มากกว่า 60 วัน,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.000000,0.0,NaN,NaN


In [334]:
# AccommodationOwner
data = {'AccommodationOwner': ['เป็นของตนเอง/คู่สมรส', 'พักอาศัยอยู่กับบิดามารดา/ญาติ', 'พักอาศัยอยู่กับผู้อื่น', 'เช่า','สถานที่อื่นๆ']}
df_main_AccommodationOwner = pd.DataFrame(data=data, ).set_index('AccommodationOwner')
AccommodationOwner = process_woe(df_main=df_main_AccommodationOwner, df=df, col='AccommodationOwner', col_gb='FlagGB_DQ60')
WOE_cal(AccommodationOwner)

,n_obs,n_good,n_bad,prop_n_obs,prop_bad,prop_good,prop_n_good,prop_n_bad,WOE,IV
เป็นของตนเอง/คู่สมรส,2367,2356,11.0,0.669969,0.003114,0.996886,0.671608,0.44,0.422900,0.097947
พักอาศัยอยู่กับบิดามารดา/ญาติ,1060,1049,11.0,0.300028,0.003114,0.996886,0.299031,0.44,-0.386228,0.054446
พักอาศัยอยู่กับผู้อื่น,52,49,3.0,0.014718,0.000849,0.999151,0.013968,0.12,-2.150718,0.228045
เช่า,18,18,0.0,0.005095,0.000000,1.000000,0.005131,0.00,inf,inf
สถานที่อื่นๆ,36,36,0.0,0.010190,0.000000,1.000000,0.010262,0.00,inf,inf


In [335]:
# Position
data = {'Position': ['บริหารระดับสูง', 'ผู้บริหารระดับกลาง', 'ผู้บริหารระดับต้น', 'พนักงานเจ้าหน้าที่','อื่นๆ']}
df_main_Position = pd.DataFrame(data=data, ).set_index('Position')
Position = process_woe(df_main=df_main_Position, df=df, col='Position', col_gb='FlagGB_DQ60')
WOE_cal(Position)

,n_obs,n_good,n_bad,prop_n_obs,prop_bad,prop_good,prop_n_good,prop_n_bad,WOE,IV
บริหารระดับสูง,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.000000,0.00,NaN,NaN
ผู้บริหารระดับกลาง,137.0,137.0,0.0,0.038788,0.000000,1.000000,0.039065,0.00,inf,inf
ผู้บริหารระดับต้น,589.0,578.0,11.0,0.166761,0.003114,0.996886,0.164813,0.44,-0.981962,0.270223
พนักงานเจ้าหน้าที่,2543.0,2532.0,11.0,0.719989,0.003114,0.996886,0.721985,0.44,0.495229,0.139647
อื่นๆ,20.0,20.0,0.0,0.005663,0.000000,1.000000,0.005703,0.00,inf,inf
ผู้บริหารระดับสูง,243.0,240.0,3.0,0.068800,0.000849,0.999151,0.068435,0.12,-0.561614,0.028960


In [336]:
#SalaryDeductionAgreement
data = {'SalaryDeductionAgreement': ['หน่วยงานมีข้อตกลงหักเงินเดือนชำระหนี้ให้ธนาคารเป็นลำดับแรกก่อนเจ้าหนี้รายอื่น', 'หน่วยงานมีข้อตกลงหักเงินเดือน', 'หน่วยงานมีข้อตกลงนำเงินสะสมเข้าบัญชีกรณีออกจากงาน หรือหน่วยงานชำระหนี้แทนถ้าผิดนัดชำระหนี้', 'โครงการมีข้อตกลงรับซื้อคืนกรณีผู้ขอสินเชื่อผิดนัดชำระหนี้']}
df_main_SalaryDeductionAgreement= pd.DataFrame(data=data, ).set_index('SalaryDeductionAgreement')
SalaryDeductionAgreement = process_woe(df_main=df_main_SalaryDeductionAgreement, df=df, col='SalaryDeductionAgreement', col_gb='FlagGB_DQ60')
WOE_cal(SalaryDeductionAgreement)

,n_obs,n_good,n_bad,prop_n_obs,prop_bad,prop_good,prop_n_good,prop_n_bad,WOE,IV
หน่วยงานมีข้อตกลงหักเงินเดือนชำระหนี้ให้ธนาคารเป็นลำดับแรกก่อนเจ้าหนี้รายอื่น,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.0,NaN,NaN
หน่วยงานมีข้อตกลงหักเงินเดือน,2678.0,2669.0,9.0,1.0,0.003361,0.996639,1.0,1.0,0.0,0.0
หน่วยงานมีข้อตกลงนำเงินสะสมเข้าบัญชีกรณีออกจากงาน หรือหน่วยงานชำระหนี้แทนถ้าผิดนัดชำระหนี้,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.0,NaN,NaN
โครงการมีข้อตกลงรับซื้อคืนกรณีผู้ขอสินเชื่อผิดนัดชำระหนี้,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.0,NaN,NaN


### ranking

In [ ]:
#ลอง plot ตามจำนวน distribution เเล้ว ค่อยเเบ่ง bining ตาม score
pd.cut